In [1]:
E = 210000000
v=0.3
t=0.025
w=3000

In [3]:
xi, yi = (0,0)
xj, yj = (0.5,0.25)
xm, ym = (0,0.25)

xp, yp = (0,0)
xq, yq = (0.5,0)
xr, yr = (0.5,0.25)

A1 = (xi*(yj-ym) + xj*(ym-yi)+xm*(yi-yj))/2
A2 = (xp*(yq-yr) + xq*(yr-yp)+xr*(yp-yq))/2

In [11]:
import numpy as np
B1 = (np.array([yj-ym,0,ym-yi,0, yi-yj,0,0,xm-xj,0,xi-xm,0,xj-xi,xm-xj,yj-ym,xi-xm,ym-yi,xj-xi,yi-yj]).reshape((3,6)))/(2*A1)
B2 = (np.array([yq-yr,0,yr-yp,0, yp-yq,0,0,xr-xq,0,xp-xr,0,xq-xp,xr-xq,yq-yr,xp-xr,yr-yp,xq-xp,yp-yq]).reshape((3,6)))/(2*A2)

D = (E/(1-v**2)) * (np.array([1,v,0,v,1,0,0,0,(1-v)/2]).reshape((3,3)))

D

array([[2.30769231e+08, 6.92307692e+07, 0.00000000e+00],
       [6.92307692e+07, 2.30769231e+08, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.07692308e+07]])